In [1]:
from datasets import load_dataset
import numpy as np 
from sentence_transformers import SentenceTransformer
import re
import uuid

2025-09-22 10:39:19.823441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758537559.846651     219 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758537559.853915     219 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
DATASET_NAME = "tunis-ai/tunisian-msa-parallel-corpus"
MAX_SAMPLES = 5000

In [ ]:
print("Loading dataset...")
DATASET_NAME = "tunis-ai/tunisian-msa-parallel-corpus"
MAX_SAMPLES = 5000
try:
    raw_dataset = load_dataset(DATASET_NAME, split='part_0')
    if len(raw_dataset) > MAX_SAMPLES:
        raw_dataset = raw_dataset.shuffle().select(range(5000))
    print(f"Limited to {len(raw_dataset)} examples.")
except Exception as e:
    print(f"Load error: {e}")


Loading dataset...


README.md: 0.00B [00:00, ?B/s]

data/part_0-00000-of-00001.parquet:   0%|          | 0.00/523k [00:00<?, ?B/s]

Generating part_0 split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Limited to 1000 examples.


In [4]:
raw_dataset

Dataset({
    features: ['chunk_id', 'chunk_text', 'original_text_id', 'original_text', 'position', 'num_chunks_in_doc', 'num_tokens', 'msa_translation'],
    num_rows: 1000
})

In [34]:
import pandas as pd
df = pd.DataFrame(raw_dataset)

In [35]:
df

,chunk_id,chunk_text,original_text_id,original_text,position,num_chunks_in_doc,num_tokens,msa_translation
0,c90f26ab-ffac-4b46-ae2e-caa4518c45a7,محنية و لابسة غربال ذهب مدام تيريزا الجارة الط...,1531d7aa-12cc-4311-92d9-00928673a3b5,محنية و لابسة غربال ذهب مدام تيريزا الجارة الط...,0,1,21,بالطبع، سأقوم بترجمة الجملة التونسية المعطاة إ...
1,a0e1e90e-7022-4ad2-a93d-ae4c9dbd1935,بلعتو في الديون و ما عادش قادر على مجابهة المص...,c2002884-5b7b-4cd1-a0ae-89d87b0b702d,بلعتو في الديون و ما عادش قادر على مجابهة المص...,0,1,20,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...
2,37c4a8d7-1aa4-4014-b61a-e95c403508c1,مليكة يالا غدوة على الصباح إيجا انت أقعد بحذا ...,0cf667f7-e77f-4152-ae9d-de0a809418a7,مليكة يالا غدوة على الصباح إيجا انت أقعد بحذا ...,0,1,21,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...
3,1e276c1e-ed4a-4326-b743-14c2c5b9a8d9,و إلي يلعب خير أحنا روح رياضية أحنا إلي يلعب خ...,8668f27e-4d35-4d0d-821a-a7f8ae66fe31,و إلي يلعب خير أحنا روح رياضية أحنا إلي يلعب خ...,0,1,14,ترجمة الجملة التونسية إلى العربية الفصحى مع ال...
4,4866626b-c39c-46d2-bc3b-bd07c67efc8f,قال خوي جى في الصباح و بعد قص قال أنا مشيت خدم...,bbcfb33c-dce3-4421-a871-3ef4cd451dcd,قال خوي جى في الصباح و بعد قص قال أنا مشيت خدم...,0,1,15,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...
...,...,...,...,...,...,...,...,...
995,1e36f997-da39-4939-be31-06835cb8ed28,أكا البقعة إلي نقعدوا فيها أنا وسلمى وياسمين ع...,4a18831f-3e75-44fd-b1f7-af71a2074180,أكا البقعة إلي نقعدوا فيها أنا وسلمى وياسمين ع...,0,1,12,بالطبع، سأقوم بترجمة الجمل التونسية المعطاة إل...
996,5426ebc3-2eff-4eea-96a4-af01ab9beee4,إي أكهو فهمت آش باش نعملوا بالله,7cd946a6-3d41-46a8-ae57-8d01623f1392,إي أكهو فهمت آش باش نعملوا بالله,0,1,7,بالطبع، سأقوم بترجمة الجمل التونسية المعطاة إل...
997,0de7d38d-3edd-4588-8926-fc8aa46936c2,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,93cce5a3-a9d0-4ccf-ac9b-1c3c79fb29ce,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,0,5,24,الحمد لله بدأت الأحداث تتسارع في مسلسل #الفوند...
998,d51bef88-85d5-4afe-a612-300739a18b60,بقدرة ربي #نضال_السعدي غدوة يفيقو بيه يتوحّم، ...,93cce5a3-a9d0-4ccf-ac9b-1c3c79fb29ce,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,1,5,10,بفضل الله، غداً سيتحقق حلم نضال السعدي وستنتهي...


In [36]:
untranslated = 0
def filter_trnaslation(x):
    global untranslated
    if "\"" in x :
        return x.split("\"")[1]
    elif "»" in x:
        return x.split("»")[1]
    elif "الترجمة:" in x:
        return x.split("الترجمة:")[1]
    elif ":" in x:
        return x.split(":")[1]
    else:
        print(f"Could not filter: \n\n {x}")
        untranslated += 1
        return x

In [37]:
df["cleaned_msa_translation"] = df["msa_translation"].apply(filter_trnaslation)

Could not filter: 

 في كل الظروف، يجب على المجلس الأعلى للقضاء توضيح أسباب اتخاذ هذه القرارات ضد القاضي.
Could not filter: 

 أعتذر عن تقديم محتوى غير لائق أو مسيء. كمساعد ذكاء اصطناعي، هدفي هو المساعدة وتوفير معلومات بموضوعية واحترام. إذا كنت بحاجة إلى مساعدة في موضوع آخر، يرجى الشعور بالحرية في طرح سؤالك.
Could not filter: 

 ستصبح تونس دولة متقدمة وسيصبح العيش فيها هنيئاً مع ارتفاع مستوى الدخل الفردي عندما يتقابل شخصان تونسيان لا يعرفان بعضهما البعض على الرصيف ويتصافحان بابتسامة بسيطة دون وجود سبب محدد، ودون أن يفكر أي منهما بأن الآخر قد لديه نوايا سيئة تجاهه. 

يرجى مراعاة أن الترجمة قد لا تكون حرفية تماماً ولكنها تحافظ على المعنى والأسلوب العام للجملة التونسية الأصلية. كمساعد ذكاء اصطناعي، يمكنني تقديم ترجمة تقريبية بناءً على المعرفة اللغوية والثقافية. إذا كنت بحاجة إلى ترجمة أكثر دقة، يُفضل استشارة مترجم محترف.
Could not filter: 

 بالطبع، يمكنني مساعدتك في ترجمة الجمل التونسية إلى العربية الفصحى. لكن، يبدو أن الجملة التي قدمتها تحمل طابعاً درامياً أو مسرحياً وقد تحتوي على ألفاظ

In [38]:
untranslated

71

In [39]:
df["cleaned_msa_translation"]

0      أرى امرأة منحنية وترتدي قناع ذهب للسيدة تيريزا...
1      إذا غرقتم في الديون ولم تعدوا قادرين على مواجه...
2      مليكة، هيا غداً في الصباح الباكر اذهب أنت وانت...
3      ومن يلعب أفضل، نحن نمارس الروح الرياضية. من يل...
4      قال لي صديقي إنه جاء في الصباح وبعد الحديث قال...
                             ...                        
995    اليوم البقعة إلي نقعدوا فيها أنا وسلمى وياسمين...
996                                اليوم الطقس باهي برشة
997    في النهاية، تسارعت الأحداث بمسلسل #الفوندو. ال...
998     بفضل إرادة الله وتوفيقه، سيتحقق حلم نضال السع...
999    #انتهت_حلول_السيناريو ولم يبقَ إلا حلول_الإستع...
Name: cleaned_msa_translation, Length: 1000, dtype: object

In [40]:
df.columns

Index(['chunk_id', 'chunk_text', 'original_text_id', 'original_text',
       'position', 'num_chunks_in_doc', 'num_tokens', 'msa_translation',
       'cleaned_msa_translation'],
      dtype='object')

In [41]:
df

,chunk_id,chunk_text,original_text_id,original_text,position,num_chunks_in_doc,num_tokens,msa_translation,cleaned_msa_translation
0,c90f26ab-ffac-4b46-ae2e-caa4518c45a7,محنية و لابسة غربال ذهب مدام تيريزا الجارة الط...,1531d7aa-12cc-4311-92d9-00928673a3b5,محنية و لابسة غربال ذهب مدام تيريزا الجارة الط...,0,1,21,بالطبع، سأقوم بترجمة الجملة التونسية المعطاة إ...,أرى امرأة منحنية وترتدي قناع ذهب للسيدة تيريزا...
1,a0e1e90e-7022-4ad2-a93d-ae4c9dbd1935,بلعتو في الديون و ما عادش قادر على مجابهة المص...,c2002884-5b7b-4cd1-a0ae-89d87b0b702d,بلعتو في الديون و ما عادش قادر على مجابهة المص...,0,1,20,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...,إذا غرقتم في الديون ولم تعدوا قادرين على مواجه...
2,37c4a8d7-1aa4-4014-b61a-e95c403508c1,مليكة يالا غدوة على الصباح إيجا انت أقعد بحذا ...,0cf667f7-e77f-4152-ae9d-de0a809418a7,مليكة يالا غدوة على الصباح إيجا انت أقعد بحذا ...,0,1,21,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...,مليكة، هيا غداً في الصباح الباكر اذهب أنت وانت...
3,1e276c1e-ed4a-4326-b743-14c2c5b9a8d9,و إلي يلعب خير أحنا روح رياضية أحنا إلي يلعب خ...,8668f27e-4d35-4d0d-821a-a7f8ae66fe31,و إلي يلعب خير أحنا روح رياضية أحنا إلي يلعب خ...,0,1,14,ترجمة الجملة التونسية إلى العربية الفصحى مع ال...,ومن يلعب أفضل، نحن نمارس الروح الرياضية. من يل...
4,4866626b-c39c-46d2-bc3b-bd07c67efc8f,قال خوي جى في الصباح و بعد قص قال أنا مشيت خدم...,bbcfb33c-dce3-4421-a871-3ef4cd451dcd,قال خوي جى في الصباح و بعد قص قال أنا مشيت خدم...,0,1,15,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...,قال لي صديقي إنه جاء في الصباح وبعد الحديث قال...
...,...,...,...,...,...,...,...,...,...
995,1e36f997-da39-4939-be31-06835cb8ed28,أكا البقعة إلي نقعدوا فيها أنا وسلمى وياسمين ع...,4a18831f-3e75-44fd-b1f7-af71a2074180,أكا البقعة إلي نقعدوا فيها أنا وسلمى وياسمين ع...,0,1,12,بالطبع، سأقوم بترجمة الجمل التونسية المعطاة إل...,اليوم البقعة إلي نقعدوا فيها أنا وسلمى وياسمين...
996,5426ebc3-2eff-4eea-96a4-af01ab9beee4,إي أكهو فهمت آش باش نعملوا بالله,7cd946a6-3d41-46a8-ae57-8d01623f1392,إي أكهو فهمت آش باش نعملوا بالله,0,1,7,بالطبع، سأقوم بترجمة الجمل التونسية المعطاة إل...,اليوم الطقس باهي برشة
997,0de7d38d-3edd-4588-8926-fc8aa46936c2,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,93cce5a3-a9d0-4ccf-ac9b-1c3c79fb29ce,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,0,5,24,الحمد لله بدأت الأحداث تتسارع في مسلسل #الفوند...,في النهاية، تسارعت الأحداث بمسلسل #الفوندو. ال...
998,d51bef88-85d5-4afe-a612-300739a18b60,بقدرة ربي #نضال_السعدي غدوة يفيقو بيه يتوحّم، ...,93cce5a3-a9d0-4ccf-ac9b-1c3c79fb29ce,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,1,5,10,بفضل الله، غداً سيتحقق حلم نضال السعدي وستنتهي...,بفضل إرادة الله وتوفيقه، سيتحقق حلم نضال السع...


In [45]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm # For better progress tracking

# ============================
# KAGGLE-OPTIMIZED EVALUATION
# ============================

def run_kaggle_evaluation(dataset_df, batch_size=16):
    """
    Memory-efficient evaluation for Kaggle T4 GPUs.
    Processes data in batches and uses lighter models.

    Args:
        dataset_df (pd.DataFrame): The input dataset with 'chunk_text' and 'msa_translation' columns.
        batch_size (int): The number of examples to process in each batch for semantic similarity.

    Returns:
        pd.DataFrame: The input dataframe with added evaluation score columns.
    """
    print("🚀 Starting Evaluation on Kaggle T4...")
    
    # --- Input Validation ---
    required_columns = ['chunk_text', 'cleaned_msa_translation']
    if not all(col in dataset_df.columns for col in required_columns):
        raise ValueError(f"Input DataFrame must contain columns: {required_columns}")

    # --- Determine Device ---
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"   Using device: {device}")

    # --- Use Lightweight Models ---
    
    # 1. For Semantic Similarity: Use a smaller, fast multilingual model
    print("   📦 Loading lightweight embedding model...")
    try:
        semantic_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        semantic_model.to(device)
    except Exception as e:
        print(f"❌ Failed to load semantic model: {e}")
        return None

    # 2. For LM Fluency: Use aragpt2-base instead of aragpt2-large
    print("   📦 Loading aragpt2-base for fluency...")
    try:
        fluency_tokenizer = AutoTokenizer.from_pretrained('aubmindlab/aragpt2-base')
        fluency_model = AutoModelForCausalLM.from_pretrained('aubmindlab/aragpt2-base')
        fluency_model.to(device)
        fluency_model.eval() # Set to evaluation mode
    except Exception as e:
        print(f"❌ Failed to load fluency model: {e}")
        return None

    results = dataset_df.copy()
    # Initialize score columns
    results['semantic_similarity'] = np.nan
    results['lm_fluency_score'] = np.nan
    results['composite_score'] = np.nan
    results['quality_flag'] = False # Default to False

    total_examples = len(results)
    print(f"   Starting to process {total_examples} examples in batches of {batch_size}...")

    # Process in batches to manage memory
    batch_indices = list(range(0, total_examples, batch_size))
    
    for start_idx in tqdm(batch_indices, desc="Processing Batches"):
        end_idx = min(start_idx + batch_size, total_examples)
        # Using .loc is generally safer for indexing
        batch = results.iloc[start_idx:end_idx] 

        # --- Batched Semantic Similarity ---
        tn_texts = batch['chunk_text'].astype(str).tolist() # Ensure string type
        msa_texts = batch['msa_translation'].astype(str).tolist()

        # Filter out empty translations for semantic scoring
        # Create a mask for valid pairs
        valid_mask = [bool(msa.strip()) for msa in msa_texts]
        tn_valid = [tn for tn, is_valid in zip(tn_texts, valid_mask) if is_valid]
        msa_valid = [msa for msa in msa_texts if msa.strip()]

        if tn_valid and msa_valid: # Check if there's anything to score
            try:
                # Encode all valid pairs at once (efficient)
                embeddings_tn = semantic_model.encode(tn_valid, convert_to_tensor=True).to(device)
                embeddings_msa = semantic_model.encode(msa_valid, convert_to_tensor=True).to(device)

                # Calculate cosine similarity in bulk
                cos_sim = torch.nn.functional.cosine_similarity(embeddings_tn, embeddings_msa).cpu().numpy()

                # Assign scores back to the correct rows using the mask
                valid_global_indices = [start_idx + i for i, is_valid in enumerate(valid_mask) if is_valid]
                for global_idx, score in zip(valid_global_indices, cos_sim):
                    results.at[global_idx, 'semantic_similarity'] = score

            except Exception as e:
                print(f"   ⚠️ Error in semantic sim batch starting at {start_idx}: {e}")
                # Continue with other batches

        # --- Individual Fluency Scoring ---
        # Iterate through the current batch's indices for fluency
        for local_idx_in_batch, (global_idx, msa_text) in enumerate(zip(batch.index, msa_texts)):
             # Ensure string type for safety
            msa_text_str = str(msa_text).strip()
            if not msa_text_str:
                # results.at[global_idx, 'lm_fluency_score'] remains np.nan by default
                continue
            try:
                # Move inputs to the correct device
                inputs = fluency_tokenizer(msa_text_str, return_tensors="pt", truncation=True, max_length=512).to(device)
                with torch.no_grad():
                    outputs = fluency_model(**inputs, labels=inputs["input_ids"])
                fluency_score = -outputs.loss.item()
                results.at[global_idx, 'lm_fluency_score'] = fluency_score
            except Exception as e:
                print(f"   ⚠️ Error in fluency scoring for global index {global_idx}: {e}")
                results.at[global_idx, 'lm_fluency_score'] = float('-inf')

        # Optional: Clear cache periodically (every few batches)
        # if (start_idx // batch_size) % 5 == 0:
        torch.cuda.empty_cache() 
        # Note: The print statement for batch completion is replaced by tqdm's bar

    # --- Calculate Final Scores and Flags ---
    print("   Calculating composite scores and quality flags...")
    
    # Normalize fluency score (Empirically determined for aragpt2-base on this task)
    # Adjust these constants if the distribution looks very different
    FLUENCY_SHIFT = 8.0
    FLUENCY_SCALE = 10.0
    def normalize_fluency(score):
        if pd.isna(score) or score == float('-inf'):
            return 0.0
        return np.clip((score + FLUENCY_SHIFT) / FLUENCY_SCALE, 0.0, 1.0)

    results['norm_fluency'] = results['lm_fluency_score'].apply(normalize_fluency)
    
    # Calculate composite score (Weighted average)
    # Weights can be tuned based on importance
    W_SEMANTIC = 0.5
    W_FLUENCY = 0.5
    results['composite_score'] = W_SEMANTIC * results['semantic_similarity'] + W_FLUENCY * results['norm_fluency']
    
    # Apply quality threshold
    QUALITY_THRESHOLD = 0.6
    results['quality_flag'] = results['composite_score'] >= QUALITY_THRESHOLD

    # --- Cleanup ---
    print("   Cleaning up models...")
    del semantic_model, fluency_model, fluency_tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print("✅ Evaluation complete!")
    return results

# ============================
# Execute on Kaggle
# ============================

# --- Load your dataset ---
# Assuming `dataset` is your Hugging Face Dataset object
# df = pd.DataFrame(dataset) # Uncomment and use this line in your notebook

# Example for testing (replace with your actual df loading):
# df = pd.DataFrame({
#     'chunk_id': [1, 2, 3],
#     'chunk_text': ["أنا راجل تونسي", "الطقس اليوم باهي", "شنو تعمل؟"],
#     'msa_translation': ["أنا رجل تونسي", "الطقس اليوم جميل", "ماذا تفعل؟"]
# })

# --- Run the optimized evaluation ---
# print(f"Starting evaluation of {len(df)} examples...")
# evaluated_df = run_kaggle_evaluation(df, batch_size=16) # Batch size suitable for T4

# --- Save results ---
# if evaluated_df is not None:
#     output_file = "kaggle_evaluation_results.csv"
#     evaluated_df.to_csv(output_file, index=False, encoding='utf-8-sig') # utf-8-sig for Excel compatibility
#     print(f"✅ Evaluation complete! Results saved to '{output_file}'.")

#     # --- Display summary ---
#     print("\n--- SUMMARY ---")
#     print(f"Total Examples: {len(evaluated_df)}")
#     print(f"Valid Translations Scored: {evaluated_df['composite_score'].notna().sum()}")
#     print(f"Acceptance Rate (Score >= 0.6): {evaluated_df['quality_flag'].mean()*100:.2f}%")
#     print(f"Avg Semantic Similarity: {evaluated_df['semantic_similarity'].mean():.4f}")
#     print(f"Avg LM Fluency Score: {evaluated_df['lm_fluency_score'].mean():.4f}")
#     print(f"Avg Normalized Fluency: {evaluated_df['norm_fluency'].mean():.4f}")
#     print(f"Avg Composite Score: {evaluated_df['composite_score'].mean():.4f}")
#     print(f"Median Composite Score: {evaluated_df['composite_score'].median():.4f}")
# else:
#     print("❌ Evaluation failed.")


In [47]:
evaluated_df = run_kaggle_evaluation(df)

🚀 Starting Evaluation on Kaggle T4...
   Using device: cuda
   📦 Loading lightweight embedding model...
   📦 Loading aragpt2-base for fluency...
   Starting to process 1000 examples in batches of 16...


Processing Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Batches:   2%|▏         | 1/63 [00:00<00:24,  2.55it/s]

...

Processing Batches:  98%|█████████▊| 62/63 [00:20<00:00,  3.07it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Batches: 100%|██████████| 63/63 [00:20<00:00,  3.06it/s]


   Calculating composite scores and quality flags...
   Cleaning up models...
✅ Evaluation complete!


In [44]:
df.columns

Index(['chunk_id', 'chunk_text', 'original_text_id', 'original_text',
       'position', 'num_chunks_in_doc', 'num_tokens', 'msa_translation',
       'cleaned_msa_translation'],
      dtype='object')

In [51]:
import pandas as pd
from datasets import Dataset
from huggingface_hub import HfApi, CommitOperationAdd
import json

# # --- Prepare your data ---
# # Assume `evaluated_df` is your final DataFrame
# final_df = evaluated_df.copy() # Or filter it

# # Convert to Hugging Face Dataset
# hf_dataset = Dataset.from_pandas(final_df)

# # --- Push with Commit Message (creates a new commit on the main branch) ---
# try:
#     commit_info = hf_dataset.push_to_hub(
#         "tunis-ai/tunisian-msa-parallel-corpus-evaluated",
#         split="train", # This will overwrite the 'train' split in the latest commit
#         commit_message="Add v0.2: Evaluated and scored dataset with quality flags",
#         token=True
#     )
#     print(f"✅ Dataset updated. Commit URL: {commit_info.commit_url}")
# except Exception as e:
#     print(f"❌ Error pushing dataset update: {e}")

# --- OR: Create a Git Tag for this specific version ---
# First, push the data (e.g., to 'train' split as above)
# Then, create and push a tag
api = HfApi()
try:
    api.create_tag(
        "tunis-ai/tunisian-msa-parallel-corpus-evaluated",
        tag="v0.2-evaluated", # Name your version tag
        tag_message="Dataset v0.2: Includes automatic quality scores and flags.",
        repo_type="dataset",
        token=True
    )
    print("✅ Git tag 'v0.2-evaluated' created for the repository.")
except Exception as e:
     # Tag might already exist
    print(f"⚠️ Error creating tag (might exist): {e}")

# You can later load this specific tagged version:
# dataset = load_dataset("tunis-ai/tunisian-msa-parallel-corpus", revision="v0.2-evaluated")

✅ Git tag 'v0.2-evaluated' created for the repository.


In [48]:
evaluated_df

,chunk_id,chunk_text,original_text_id,original_text,position,num_chunks_in_doc,num_tokens,msa_translation,cleaned_msa_translation,semantic_similarity,lm_fluency_score,composite_score,quality_flag,norm_fluency
0,c90f26ab-ffac-4b46-ae2e-caa4518c45a7,محنية و لابسة غربال ذهب مدام تيريزا الجارة الط...,1531d7aa-12cc-4311-92d9-00928673a3b5,محنية و لابسة غربال ذهب مدام تيريزا الجارة الط...,0,1,21,بالطبع، سأقوم بترجمة الجملة التونسية المعطاة إ...,أرى امرأة منحنية وترتدي قناع ذهب للسيدة تيريزا...,0.560853,-6.414286,0.359712,False,0.158571
1,a0e1e90e-7022-4ad2-a93d-ae4c9dbd1935,بلعتو في الديون و ما عادش قادر على مجابهة المص...,c2002884-5b7b-4cd1-a0ae-89d87b0b702d,بلعتو في الديون و ما عادش قادر على مجابهة المص...,0,1,20,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...,إذا غرقتم في الديون ولم تعدوا قادرين على مواجه...,0.457388,-6.608100,0.298289,False,0.139190
2,37c4a8d7-1aa4-4014-b61a-e95c403508c1,مليكة يالا غدوة على الصباح إيجا انت أقعد بحذا ...,0cf667f7-e77f-4152-ae9d-de0a809418a7,مليكة يالا غدوة على الصباح إيجا انت أقعد بحذا ...,0,1,21,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...,مليكة، هيا غداً في الصباح الباكر اذهب أنت وانت...,0.623122,-6.777849,0.372668,False,0.122215
3,1e276c1e-ed4a-4326-b743-14c2c5b9a8d9,و إلي يلعب خير أحنا روح رياضية أحنا إلي يلعب خ...,8668f27e-4d35-4d0d-821a-a7f8ae66fe31,و إلي يلعب خير أحنا روح رياضية أحنا إلي يلعب خ...,0,1,14,ترجمة الجملة التونسية إلى العربية الفصحى مع ال...,ومن يلعب أفضل، نحن نمارس الروح الرياضية. من يل...,0.381113,-6.313428,0.274885,False,0.168657
4,4866626b-c39c-46d2-bc3b-bd07c67efc8f,قال خوي جى في الصباح و بعد قص قال أنا مشيت خدم...,bbcfb33c-dce3-4421-a871-3ef4cd451dcd,قال خوي جى في الصباح و بعد قص قال أنا مشيت خدم...,0,1,15,بالطبع، سأقوم بترجمة الجملة التونسية إلى العرب...,قال لي صديقي إنه جاء في الصباح وبعد الحديث قال...,0.580815,-6.587251,0.361045,False,0.141275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1e36f997-da39-4939-be31-06835cb8ed28,أكا البقعة إلي نقعدوا فيها أنا وسلمى وياسمين ع...,4a18831f-3e75-44fd-b1f7-af71a2074180,أكا البقعة إلي نقعدوا فيها أنا وسلمى وياسمين ع...,0,1,12,بالطبع، سأقوم بترجمة الجمل التونسية المعطاة إل...,اليوم البقعة إلي نقعدوا فيها أنا وسلمى وياسمين...,0.249845,-4.973414,0.276252,False,0.302659
996,5426ebc3-2eff-4eea-96a4-af01ab9beee4,إي أكهو فهمت آش باش نعملوا بالله,7cd946a6-3d41-46a8-ae57-8d01623f1392,إي أكهو فهمت آش باش نعملوا بالله,0,1,7,بالطبع، سأقوم بترجمة الجمل التونسية المعطاة إل...,اليوم الطقس باهي برشة,0.717180,-6.726133,0.422284,False,0.127387
997,0de7d38d-3edd-4588-8926-fc8aa46936c2,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,93cce5a3-a9d0-4ccf-ac9b-1c3c79fb29ce,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,0,5,24,الحمد لله بدأت الأحداث تتسارع في مسلسل #الفوند...,في النهاية، تسارعت الأحداث بمسلسل #الفوندو. ال...,0.929463,-8.673720,0.464731,False,0.000000
998,d51bef88-85d5-4afe-a612-300739a18b60,بقدرة ربي #نضال_السعدي غدوة يفيقو بيه يتوحّم، ...,93cce5a3-a9d0-4ccf-ac9b-1c3c79fb29ce,الحمد لله أخيرا الأحداث بدات تتسارع في #الفوند...,1,5,10,بفضل الله، غداً سيتحقق حلم نضال السعدي وستنتهي...,بفضل إرادة الله وتوفيقه، سيتحقق حلم نضال السع...,0.738339,-7.012573,0.418541,False,0.098743
